<a href="https://colab.research.google.com/github/Tinopino/introduction-to-github/blob/main/WorkingLlama3_1_8bmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers accelerate torch bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [5]:
import torch
import transformers


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
model_path = "/content/drive/MyDrive/llama-8b-raw/checkpoints/Llama3.1-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [1]:
def format_chat_prompt(history):
    prompt = ""
    for turn in history:
        prompt += f"<|start_header_id|>{turn['role']}<|end_header_id|>\n{turn['content']}\n"
    prompt += "<|start_header_id|>assistant<|end_header_id|>\n"
    return prompt

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
pipe = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    )

print("✅ Model loaded successfully")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
def format_chat_prompt(history):
    prompt = ""
    for turn in history:
        prompt += f"<|start_header_id|>{turn['role']}<|end_header_id|>\n{turn['content']}\n"
    prompt += "<|start_header_id|>assistant<|end_header_id|>\n"
    return prompt

def generate_response_from_text(prompt_file_path, temperature=0.7):
    # Read prompt from file
    with open(prompt_file_path, "r", encoding="utf-8") as f:
        user_input = f.read().strip()
    print(user_input)
    # Create chat history
    history = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_input}
    ]

    prompt = format_chat_prompt(history)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    generation_config = transformers.GenerationConfig(
        temperature=temperature,
        top_p=0.9,
        max_new_tokens=256,
        do_sample=True,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )

    with torch.no_grad():
        outputs = model.generate(**inputs, generation_config=generation_config)

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract assistant reply
    if "<|start_header_id|>assistant<|end_header_id|>\n" in output_text:
        assistant_reply = output_text.split("<|start_header_id|>assistant<|end_header_id|>\n")[-1].strip()
    else:
        assistant_reply = output_text.strip()

    return assistant_reply

In [ ]:
def generate_response_from_text2(prompt_file_path, temperature=0.7):
    pipe = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    )

    system_prompt = (
        "You are a skilled medical doctor reviewing clinical text. "
        "Return ERROR: 0 if correct. If not, return ERROR: 1, the sentence ID, reason, and correction as instructed."
    )

    user_prompt = """The following is the text you have to review:
    0 A 23-year-old man comes to the physician because of a painless swelling on the left side of his jaw for 2 months.
    1 It has been progressively increasing in size and is draining thick, foul-smelling fluid.
    2 He had a molar extracted 3 months ago.
    3 Examination shows a 4-cm, tender, erythematous mass in the left submandibular region with purulent drainage.
    4 There is submandibular lymphadenopathy.
    5 A culture of the purulent material shows catalase-negative, gram-positive filamentous rods that do not stain with carbol fuchsin.
    6 Culture confirms Nocardia asteroides.

    Please check the text according to the instructions. Your answer should look like this:

    ERROR: {error_tag}
    SENTENCE: {sentence_id}
    REASON: {reason_correction}
    CORRECTION: {corrected_sentence}
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = pipe(
        messages,
        max_new_tokens=100,
        do_sample=True,
        return_full_text=False
    )

    print(response[0]["generated_text"])


In [ ]:
response = generate_response_from_text2("/content/prompt.txt",temperature=0.7)
print(response)


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


ERROR: 1
SENTENCE: 5
REASON: The description of the bacteria as "catalase-negative, gram-positive filamentous rods that do not stain with carbol fuchsin" is incorrect for Nocardia asteroides, which is actually catalase-positive and gram-positive but does not form rods, rather branching filaments. 
CORRECTION: 5 A culture of the purulent material shows catalase-positive, gram-positive filamentous branching bacteria that do not stain
None
